In [ ]:
import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(os.path.dirname(current_dir))
sys.path.insert(0, parent_dir) 

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from Evaluation import get_metrics
%load_ext autoreload
%autoreload 2

---
## Set Path

In [2]:
# path = 'C:/Users/robin.opdam/Google Drive/Thesis (Msc)/Thesis_shared_files/'
# path = '/Users/Robin/Google Drive/Thesis (Msc)/Thesis_shared_files/'
path = '../'

## Set params based on top 3 GS results
- ML: Final run for GS iloc 52
- Am_like_ML: Final run for GS iloc 103
- Am_20k_users: Final run for GS iloc 106

In [3]:
file_names = ['ml_1m', 'am_like_ml', 'am_20k_users']

In [5]:
pd.read_pickle(path + 'Logs/GS/' + file_names[0] + '_cfrnn_gs_log').sort_values('val_rec@10', ascending=False).rnn_units.unique()

array([20], dtype=object)

In [5]:
ml_1m_params = {
'epochs':100,
'BATCH_SIZE':16,
'learning_rate':0.2,
'delta':0.01,             # Diversity Bias
'max_seq_len':30,        # Max length of sequence71=median
'embedding_dim':100,
'rnn_units':20,
'ckpt_dir': '',#path + f'Logs/ckpts/cfrnn_ckpts_{file_name}/ckpts',

'pad_value':3706,#total_items, # Pad with total_items+1 => masked => still use item 0
'shift_targets_by':1     
}

In [6]:
pd.read_pickle(path + 'Logs/GS/' + file_names[1] + '_cfrnn_gs_log').sort_values('val_rec@10', ascending=False).iloc[:3]

,all_val_rec@10,val_rec@10,total_val_rec,train_time,loss,val_loss,epochs,BATCH_SIZE,learning_rate,delta,max_seq_len,embedding_dim,rnn_units,ckpt_dir,test_users,val_users,pad_value,shift_targets_by
190,NaN,0.046,0.164,130.340483,"[8.70527331488473, 8.700457750047956, 8.695642...","[9.05406379699707, 9.049371310642787, 9.044669...",20,64,0.1,0.01,30,100,50,../Logs/GS/ckpts/cfrnn_ckpts_am_like_ml/ckpts_...,1000,500,86843,1
114,NaN,0.046,0.162,126.638872,"[8.70527037211827, 8.700460815429688, 8.695657...","[9.054061753409249, 9.04937676021031, 9.044692...",20,64,0.1,0.01,10,100,20,../Logs/GS/ckpts/cfrnn_ckpts_am_like_ml/ckpts_...,1000,500,86843,1
119,NaN,0.044,0.176,125.962825,"[2.314343367304121, 2.3113559212003434, 2.3083...","[5.4310590880257745, 5.428088460649763, 5.4251...",20,64,0.1,0.20,30,100,20,../Logs/GS/ckpts/cfrnn_ckpts_am_like_ml/ckpts_...,1000,500,86843,1


In [7]:
am_like_ml_params = {
'epochs':20,
'BATCH_SIZE':64,
'learning_rate':0.1,
'delta':0.01,             # Diversity Bias
'max_seq_len':30,        # Max length of sequence71=median
'embedding_dim':100,
'rnn_units':50,
'ckpt_dir': '', #path + f'Logs/ckpts/cfrnn_ckpts_{file_name}/ckpts',

'pad_value':86843, 
'shift_targets_by':1     
}

In [8]:
pd.read_pickle(path + 'Logs/GS/' + file_names[2] + '_cfrnn_gs_log').sort_values('val_rec@10', ascending=False).iloc[:3]

,all_val_rec@10,val_rec@10,total_val_rec,train_time,loss,val_loss,epochs,BATCH_SIZE,learning_rate,delta,max_seq_len,embedding_dim,rnn_units,ckpt_dir,test_users,val_users,pad_value,shift_targets_by
108,NaN,0.005,0.019,159.164211,"[1.657568097114563, 1.6558749675750732, 1.6541...","[3.894928216934204, 3.8934335708618164, 3.8919...",20,32,0.10,0.20,20,100,20,../Logs/GS/ckpts/cfrnn_ckpts_am_20k_users/ckpt...,1000,500,90395,1
99,NaN,0.004,0.017,160.078607,"[6.8912458419799805, 6.889159679412842, 6.8870...","[7.082394123077393, 7.080687046051025, 7.07899...",20,32,0.05,0.01,10,100,20,../Logs/GS/ckpts/cfrnn_ckpts_am_20k_users/ckpt...,1000,500,90395,1
102,NaN,0.004,0.016,159.416193,"[1.6579798460006714, 1.6571325063705444, 1.656...","[3.8956804275512695, 3.894932270050049, 3.8941...",20,32,0.05,0.20,20,100,20,../Logs/GS/ckpts/cfrnn_ckpts_am_20k_users/ckpt...,1000,500,90395,1


In [9]:
am_20k_users_params = {
'epochs':20,
'BATCH_SIZE':32,
'learning_rate':0.1,
'delta':0.2,             # Diversity Bias
'max_seq_len':20,        # Max length of sequence71=median
'embedding_dim':100,
'rnn_units':20,
'ckpt_dir': '',

'pad_value':90395, # Pad with total_items+1 => masked => still use item 0
'shift_targets_by':1     
}

---
# CFRNN Final Runs

In [10]:
tf.random.set_seed(5632711)

In [11]:
data_paths = ['Data/ML/', 'Data/Amazon/', 'Data/Amazon/']
params = [ml_1m_params, am_like_ml_params, am_20k_users_params]

n_runs = [30, 30, 30]

steps = 5
rank_at = 20

In [ ]:
from CFRNN import CFRNN
from Data_prep import leave_last_out
import time

for data_path, file_name, cfrnn_params, n_run in zip(data_paths, file_names, params, n_runs):
    if n_run > 0:
        df = pd.read_pickle(path + data_path + file_name)
        total_items = df.item_id.nunique()
        total_users = df.user_id.nunique()

        # Load Train Test Val Split
        train_set_cfrnn = pd.read_pickle(path + data_path + file_name + '_train_cfrnn')
        val_set_cfrnn = pd.read_pickle(path + data_path + file_name + '_val_cfrnn')
        test_set_cfrnn = pd.read_pickle(path + data_path + file_name + '_test_cfrnn')
        cfrnn = CFRNN(total_users, total_items, cfrnn_params)
        train_dataset = cfrnn.create_seq_batch_tf_dataset(train_set_cfrnn, stats=False)
        val_dataset = cfrnn.create_seq_batch_tf_dataset(val_set_cfrnn, stats=False)

        # Create val_set and test_set for prediction procedures
        val_set, left_out_val = leave_last_out(val_set_cfrnn, n_items=1)
        test_set, left_out_test = leave_last_out(test_set_cfrnn, n_items=1)
    
    for i in range(n_run):
        print(f'run: {i}')
        cfrnn_params['ckpt_dir'] = path + 'Logs/final/ckpts/cfrnn_ckpts_' + file_name + '_run_' + str(i) + '/ckpts_{epoch}'
        cfrnn = CFRNN(total_users, total_items, cfrnn_params)
        cfrnn.build_model(summary=False)
        cfrnn.compile_model(train_set=train_set_cfrnn)
                                
        s = time.time()
        cfrnn.train(train_dataset, val_dataset, verbose=0)
        
        additional_info = {'all_val_rec@10':[], 'all_val_ndcg@10':[], 'val_metrics':[]}
        for e in range(1, cfrnn_params['epochs'], 1):
            ranked_df_cfrnn = cfrnn.get_predictions(train_set_cfrnn, val_set, left_out_val, batch_size=500, rank_at=rank_at, 
                                                    ckpt_dir=f'{path}Logs/final/ckpts/cfrnn_ckpts_{file_name}_run_{i}/ckpts_{e}')
            cfrnn_metrics = get_metrics(ranked_df_cfrnn, steps, rank_at, stats=False)
            additional_info['all_val_rec@10'].append(cfrnn_metrics['recall'].iloc[2]) 
            additional_info['all_val_ndcg@10'].append(cfrnn_metrics['ndcg'].iloc[2])
            additional_info['val_metrics'].append(cfrnn_metrics)
        t = time.time() - s
        ranked_df_cfrnn = cfrnn.get_predictions(train_set_cfrnn, test_set, left_out_test, batch_size=500, rank_at=rank_at, 
                                                    ckpt_dir=f'{path}Logs/final/ckpts/cfrnn_ckpts_{file_name}_run_{i}/ckpts_{e}')
        cfrnn_test_metrics = get_metrics(ranked_df_cfrnn, steps, rank_at, stats=False)
        additional_info['metrics'] = cfrnn_test_metrics
        additional_info['train_time'] = t

        store_path = path + '/Logs/final/' + file_name +  '_cfrnn_log'
        cfrnn.store_model(store_path, additional_info)


In [ ]:
res_path = path + '/Logs/final/' + file_names[0] + '_cfrnn_log'
pd.read_pickle(res_path)

In [ ]:
# from Evaluation import get_final_results
# res_path = path + '/Logs/final/' + file_names[1] + '_bpr_log'
# final_metrics, train_stats, train_time = get_final_results(res_path)

In [ ]:
# from visualize_results import plot_final_metrics
# all_final_results = [final_metrics]
# colors = ['blue']
# labels = ['BPR']
# metrics_to_show = ['recall', 'ndcg']
# add_to_title = 'MovieLens 1m'
# plot_final_metrics(all_final_results, colors, labels, metrics_to_show, add_to_title)